# Space weather report

## Global Ionospheric Maps

We use this notebook to collect data and produce "standard" data processing and visualization to estimate space weather conditions at particular date range. By standard we mean the products that have well understood (at least for us) creteria and points that we can include in text report. The visualization should support our finding and should have all labels that are included in report. 

## Data sources

We try to use proxy to the sources providing original data since. By doing so we can balance load and have data format that we find more convinient for us and avoid code that serves as parser in current notebook. We provide link for original data sources to to give a credits and to have one more point of control and testing (since our services could also have errors). We provide the links to the data serivices we use and links to original data sources, one is free to use any of them, however **don't forget to cite original data sources in any case**:

 * [SIMuRG](https://simurg.space/) - GNSS TEC and TEC-based data products. SIMuRG processed data distributed as RINEX files from multiple [sources](https://simurg.space/sources). While TEC is not directly contained in RINEX files, they are used to calculate it. SIMuRG also provide access to Global Ionospheric Maps, which originally provide by several ionospheric centers **PROVIDE LIST**.

## Install required packages

### NOTE the notebook is tested with python version 3.10. Make sure that you user proper Pyhton kernel

We will use particular libraries that we `import` they are installed with `pip`. Feel free to another one to the cell above if you modify code.

In [ ]:
!pip install requests
!pip install matplotlib
!pip install imageio[ffmpeg]
!pip install -e git+https://github.com/gnss-lab/ionex.git#egg=ionex
!conda install -y cartopy

## Extra libs for Windows users

Since Windows doesn't have build-in `gzip` (at least recently) and SIMuRG provide comppressed files `.Z` we need extra library that will uncompress files. 

In [ ]:
!pip install unlzw3

In [1]:
try:
    import ionex
except:
    raise ValueError("Check output of previous cell. And restart Kernel if all installed")

## Logger

We also define logger to have better structured messages and more control over visibility of message than with just `print`. Change `logger.setLevel(logging.DEBUG)` to `logger.setLevel(logging.INFO)` to reduce number of log messages. 

In [73]:
import logging

logger = logging.getLogger("SWReport")
logger.handlers.clear()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

## Let's start

We try to keep this notebook as clean as possible. While some functions could be implemented directly in a cell, the other ones (if they are huge) could be moved to separate modules that we keep along with notebook. We should define some properties to define what exectly we what to obtain and using which data. Make sure that don't change this constant since it will affect all cells executed further. If you need to change any of them make sure you do it carefully. Finally we define types that we use to annotate arguments of the function and its output. 

**The constants defined in the cell below will define behaviour of calculation. For example you can define the time interval of interest and reference date using which we calculate deltas**

In [98]:
from datetime import datetime, timezone
from pathlib import Path
import os
import sys
import imageio.v2 as imageio
from numpy.typing import NDArray

# constants to keep settings
PREFER_GIM = "uqrg"
STUDY_INTERVAL_START = datetime(2024, 5, 15, tzinfo=timezone.utc)
STUDY_INTERVAL_END = datetime(2024, 5, 20, tzinfo=timezone.utc)
REFERENCE_DATE = datetime(2024, 5, 15, tzinfo=timezone.utc)
MAP_DTYPE = [('lat', 'float'), ('lon', 'float'), ('vals', 'float')]
GIM_LAT_DIMENSION = 71
GIM_LON_DIMENSION = 73
CBAR_LIMITS_DELTA = (-50, 50)
CBAR_LIMITS_MAPS = (0, 200)
DPI = 100
GIM_PLOTS_PATH = Path("./gim_plot")
GIM_IONEX_PATH = Path("./gim_ionex")
os.makedirs(GIM_PLOTS_PATH, exist_ok=True)
os.makedirs(GIM_IONEX_PATH, exist_ok=True)

PLOTTING_PARAMS = {
    'font.size': 20,
    'figure.dpi': DPI,
    'font.family': 'serif',
    #'font.family': 'monospace',
    'font.style': 'normal',
    'font.weight': 'light',
    'legend.frameon': True,
    'font.variant' : 'small-caps',
    'axes.titlesize' : 20,
    'axes.labelsize' : 20,
    'xtick.labelsize' : 18,                         
    'xtick.major.pad': 5,
    'ytick.major.pad': 5,   
    'xtick.major.width' : 2.5,
    'ytick.major.width' : 2.5,
    'xtick.minor.width' : 2.5,
    'ytick.minor.width' : 2.5,
    'ytick.labelsize' : 20,
}


# Global ionospheric maps 

Global ionospheric maps are based on GNSS data and provide global coverage with 2.5 x 5 degree resolution by latitude and longitude accordingly. The timestep is diffetent for different data source and could be in range from 2 hours up to 15 minutes. Each file containce data for single day. We use SIMuRG API to query and download IONEX files where GIMs are stored. 

In [78]:
from requests import Session
from datetime import date, timedelta
from io import BytesIO

session = Session()
current = STUDY_INTERVAL_START
ionex_compressed = {}
while current <= STUDY_INTERVAL_END:
        
    response = session.get(
        "https://api.simurg.space/datafiles/gim_list", 
        params=(("d", current.date()), )
    )
    
    if response.status_code != 200:
        raise RuntimeError("Failed to get GIM source list {response.status_code} {response.reason}")
    
    gims_sources = response.json()
    
    logger.debug(f"For {current.date()} GIMs are available from ionospheric centers: {gims_sources}")
    
    if PREFER_GIM in gims_sources:
        r = session.get(
            "https://api.simurg.space/datafiles/gim", 
            params=(("d", current.date()), ("gim_type", PREFER_GIM))
        )
        if r.status_code != 200:
            raise RuntimeError(f"Could not query {PREFER_GIM} status code {r.status_code} and reason {r.reason}")
        content_details = r.headers["content-disposition"]
        content_type = r.headers.get("content-type")
        logger.debug(f"Reponse came with details: {content_details}")
        logger.debug(content_type)
        filename = GIM_IONEX_PATH / content_details.replace("attachment; filename=", "").replace('"', '')
        with open(filename, "wb") as f:
            f.write(BytesIO(r.content).read())
        ionex_compressed[current.date()] = filename
    else:
        logger.warning(f"Could not find {PREFER_GIM} for {current.date()}")
        ionex_compressed[current.date()] = None
    current = current + timedelta(1)
dates = [str(d) for d in ionex_compressed]
logger.info(f"Obtained data for {dates}. See message above for missing data")

2024-06-13 05:08:35,184 - SWReport - DEBUG - For 2024-05-15 GIMs are available from ionospheric centers: ['iri16', 'cod0', 'esa0', 'esa0', 'esa0', 'igs0', 'casg', 'uhrg', 'upcg', 'uqrg', 'cod0', 'igs0', 'cas0', 'emr0', 'jpl0', 'upc0', 'c1pg', 'c2pg', 'carg']
2024-06-13 05:08:35,590 - SWReport - DEBUG - Reponse came with details: attachment; filename="uqrg1360.24i.Z"
2024-06-13 05:08:35,592 - SWReport - DEBUG - application/octet-stream
2024-06-13 05:08:35,607 - SWReport - DEBUG - For 2024-05-16 GIMs are available from ionospheric centers: ['cod0', 'esa0', 'esa0', 'igs0', 'uhrg', 'upcg', 'uqrg', 'casg', 'iri16', 'esa0', 'cod0', 'igs0', 'cas0', 'emr0', 'jpl0', 'upc0', 'c1pg', 'c2pg', 'carg']
2024-06-13 05:08:35,932 - SWReport - DEBUG - Reponse came with details: attachment; filename="uqrg1370.24i.Z"
2024-06-13 05:08:35,934 - SWReport - DEBUG - application/octet-stream
2024-06-13 05:08:35,958 - SWReport - DEBUG - For 2024-05-17 GIMs are available from ionospheric centers: ['cod0', 'esa0', 

## Uncompress maps (unarchive)

We don't store uncompressed maps on the disk, instead we keep it in memory as `StringIO` objects. One could always save maps on the disk, by saving StringIO object as file content. Uncompress method depends on OS. The reason for that is that [unlwz3](https://github.com/scivision/unlzw3) library seems not very popular, hence for Linux `gzip` run via `subproccess` is preffered.

In [106]:
import subprocess
from io import StringIO

if sys.platform == "win32":
    import unlzw3

ionex_content = {}
for ionex_date, filename in ionex_compressed.items():
    if (sys.platform == "linux" or sys.platform == "linux2"):
        process = subprocess.Popen(('gzip', '-cdk', filename), stdout=subprocess.PIPE)
        sio = StringIO(process.stdout.read().decode(errors="ignore"))
    elif sys.platform == "win32":
        with open(filename, "rb") as compressed:
            sio = StringIO(unlzw3.unlzw(compressed.read()).decode())
        ionex_content[ionex_date] = sio
    logger.info(f"Uncompressed {ionex_date}")

2024-06-13 07:50:22,123 - SWReport - INFO - Uncompressed 2024-05-15
2024-06-13 07:50:22,153 - SWReport - INFO - Uncompressed 2024-05-16
2024-06-13 07:50:22,184 - SWReport - INFO - Uncompressed 2024-05-17
2024-06-13 07:50:22,216 - SWReport - INFO - Uncompressed 2024-05-18
2024-06-13 07:50:22,250 - SWReport - INFO - Uncompressed 2024-05-19
2024-06-13 07:50:22,282 - SWReport - INFO - Uncompressed 2024-05-20


## Read maps

Above we obtain content of IONEX files stored in StringIO obkects. Each file contains number of maps, which depends on timestep. We use [ionex](https://github.com/gnss-lab/ionex) module to read IONEX files. Some files don't obey ionex standard and there is no proper file end in them. So `"Unexpected end of the file" warning` could arise and can be safelly ignored. We need convert file contents to proper object to be able to plot them futher.

The function we will obtain maps as `numpy` arrays structures as follows.

```
array = [
    (lat1, lon1, val1)
    (lat2, lon2, val2)

     ....

    (latN, lonN, valN)
]
```

Where N is the number of grid cells, 5173 for 71x73 grid. We keep data as single array to make it easy to use it further. One can make `slice` to obtain data:

```
lats = array[:, 0]
lons = array[:, 1]
values = array[:, 2]
```


In [105]:
import ionex
import numpy as np
from io import StringIO, TextIOWrapper

gims_data = {}

def ionex_to_map2d(file: str | Path | StringIO):
    if isinstance(file, StringIO):
        file_io= file
        file_io.seek(0)
    else:
        file_io = open(file)
    try:
        for ionex_map in ionex.reader(file_io):
            data = []
            lats = ionex_map.grid.latitude
            lons = ionex_map.grid.longitude
            tec_iter = iter(ionex_map.tec)
            lon_grid = np.arange(lons.lon1, lons.lon2 + lons.dlon, lons.dlon)
            lat_grid = np.arange(lats.lat1, lats.lat2 + lats.dlat, lats.dlat)
            for lat in lat_grid:
                for lon in lon_grid:
                    data.append((lat, lon, next(tec_iter)))
            yield np.array(data, dtype=MAP_DTYPE), ionex_map.epoch.replace(tzinfo=timezone.utc)
    except ValueError as e:
        logger.warning(f"Got error '{str(e)}' when reading filename")

for ionex_date, content in ionex_content.items():
    if content is None:
        continue
    logger.debug(f"Processing {ionex_date}")
    for data, epoch in ionex_to_map2d(content):
        gims_data[epoch] = data

2024-06-13 07:49:53,277 - SWReport - DEBUG - Processing 2024-05-15
2024-06-13 07:49:53,592 - SWReport - DEBUG - Processing 2024-05-16
2024-06-13 07:49:53,876 - SWReport - DEBUG - Processing 2024-05-17
2024-06-13 07:49:54,160 - SWReport - DEBUG - Processing 2024-05-18
2024-06-13 07:49:54,478 - SWReport - DEBUG - Processing 2024-05-19
2024-06-13 07:49:54,764 - SWReport - DEBUG - Processing 2024-05-20


## Plotting layout

We define some features as geographical objects to place as well as position of axis tick and others. Function `prepare_geographical_layout()` uses constants we defined above.

In [83]:
import matplotlib
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from typing import Tuple, List
from cartopy import feature

def prepare_geographical_layout(
    ax: matplotlib.pyplot.axes, 
    lon_limits: Tuple[float],
    lat_limits: Tuple[float],
    lon_locator: List[float] = [],
    lat_locator: List[float] = []
) -> None:
    """Takes matplotlib.pyplot.axes puts landmarks and formatting
    
    
    :param lon_limits: min and max value for longitude grid
    :type lon_limits: Tuple[float]

    :param lat_limits: min and max value for latitude grid
    :type lat_limits: Tuple[float]
    """
    plt.rcParams.update(PLOTTING_PARAMS)
    gl = ax.gridlines(linewidth=2, color='gray', alpha=0.5, draw_labels=True, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    if lon_locator:
        gl.xlocator = mticker.FixedLocator(lon_locator)
    ax.set_xlim(*lon_limits)
    ax.set_ylim(*lat_limits)
    #put some features on the map
    ax.add_feature(feature.COASTLINE, linewidth=2.5)
    ax.add_feature(feature.BORDERS, linestyle=':', linewidth=2)
    ax.add_feature(feature.LAKES, alpha=0.5)
    ax.add_feature(feature.RIVERS)

## Plot maps 

We can plot maps to make sure everything is correct. We use `cartopy` to have coastlines, different projection and all other nice features. Since the GIMs are global we plot with latitude (y axis) and longitude (x axis) limits set to (-87.5, 87.5) and (-180, 180) accordingly. The poles -90 and 90 degrees by latitude are generaly not included in the GIMs since expansion functions diverge at poles and longitude has no meaning at poles.

In [84]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pathlib import Path

plt.rcParams.update(PLOTTING_PARAMS)
def plot_map(
    maps: dict[datetime, NDArray], 
    reference_date: datetime,
    savefig_to: Path | None = None,
    title: str | None = None,
    cbar_lims: tuple | None = None
):
    data = maps[epoch]
    map_array = data["vals"].reshape(GIM_LAT_DIMENSION, GIM_LON_DIMENSION)
    plt.figure(figsize=(18, 9), dpi=DPI)
    ax = plt.axes(projection=ccrs.PlateCarree())
    prepare_geographical_layout(ax, (-180, 180), (-90, 90))
    #ax.set_global()
    #ax.coastlines()
    image_ext = (-180, 180, -87.5, 87.5)
    if cbar_lims:
        image = ax.imshow(map_array, cmap="jet", extent=image_ext, vmin=cbar_lims[0], vmax=cbar_lims[1])
    else:
        image = ax.imshow(map_array, cmap="jet", extent=image_ext)
    cax = ax.figure.add_axes([ax.get_position().x1+0.01, ax.get_position().y0, 0.02, ax.get_position().height])
    cbar = ax.figure.colorbar(image, cax=cax, orientation='vertical')
    cbar.ax.set_ylabel("TEC, TECU", rotation=-90, va="bottom")
    #plt.tight_layout()
    if title is None:
        ax.set_title(f"Data from {PREFER_GIM.upper()} for {epoch} UT")
    else:
        ax.set_title(title)
    if savefig_to:
        plt.savefig(savefig_to) 
    else:
        plt.show()
    plt.close()

epochs = [epoch for epoch in gims_data]
plot_map(gims_data, epochs[96 * 2], title=f"GIM from {PREFER_GIM.upper()} \n for {epochs[0]} UT")


# Plotting and saving to files

We use function defined above to finally use GIM arrays and save them to the files. We need save to files to further pass it to procedure that will make animation out of them. When cell is executed you should see progeress as filename currently saved to the disk.

In [85]:
import sys

map_filenames = []
logger.info(f"Saving separate plots to files. See progress:")
for epoch in epochs:
    if epoch < REFERENCE_DATE:
        continue
    fname = f"map_{str(epoch)}.png".replace(":", "-") # replace to get proper windows names
    map_fname = GIM_PLOTS_PATH / fname
    title = f"GIM from {PREFER_GIM.upper()} for {epoch} UT"
    plot_map(gims_data, epoch, cbar_lims=CBAR_LIMITS_MAPS, savefig_to=map_fname, title=title)
    map_filenames.append(map_fname)
    sys.stdout.write(f"\rSaving {map_fname}")    
    sys.stdout.flush()
logger.info("Finished plotting maps")

2024-06-13 05:09:23,921 - SWReport - INFO - Saving separate plots to files. See progress:


Saving gim_plot/map_2024-05-21 00:00:00+00:00.png

2024-06-13 05:11:44,336 - SWReport - INFO - Finished plotting maps


In [86]:
logger.info(f"Make animation out of {map_filenames[0]} {map_filenames[-1]}")
with imageio.get_writer(GIM_PLOTS_PATH / f"maps_{PREFER_GIM}.gif", mode="I", loop=5) as writer:
    for filename in map_filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        sys.stdout.write(f"\rAnimate {filename}")    
        sys.stdout.flush()

2024-06-13 05:13:48,369 - SWReport - INFO - Make animation out of gim_plot/map_2024-05-15 00:00:00+00:00.png gim_plot/map_2024-05-21 00:00:00+00:00.png


Animate gim_plot/map_2024-05-21 00:00:00+00:00.png

## Display animation

Now animation is ready. You can view it in proper player or display it right here using `IPython.display`.

In [87]:
from IPython.display import display, HTML
gif_path = GIM_PLOTS_PATH / f"maps_{PREFER_GIM}.gif"
html_code = f'<img src="{gif_path}" autoplay loop>'
display(HTML(html_code))

# Delta maps

For illustrative purpose we may want to find delta maps so we can see storm events

In [88]:
def get_delta_gims(
    maps: dict[datetime, NDArray], 
    reference_date: date,
    start_epoch: datetime | None = None
)-> dict[datetime, NDArray]:
    """ Converts maps into delta maps using map for particular time from reference date"""
    delta_maps = {}
    for epoch, data in maps.items():
        if start_epoch and start_epoch > epoch:
            continue
        reference_epoch = datetime.combine(reference_date, epoch.time()).replace(tzinfo=timezone.utc)
        if not reference_epoch in maps:
            raise ValueError(f"Could not find reference for {epoch} map from epoch {reference_epoch}")
        delta_maps[epoch] = np.recarray.copy(maps[epoch])
        delta_maps[epoch]["vals"] = maps[epoch]["vals"] - maps[reference_epoch]["vals"]
    return delta_maps

In [89]:
delta_maps = get_delta_gims(gims_data, REFERENCE_DATE, start_epoch=STUDY_INTERVAL_START)
epochs = [epoch for epoch in delta_maps]

filenames = []
logger.info("Start plotting and saving delta maps")
for epoch in epochs:
    fname = f"delta_{str(epoch)}_refdate_{str(REFERENCE_DATE.date())}.png"
    fname = GIM_PLOTS_PATH / fname.replace(":", "-") # replace to get proper windows names
    title = f"GIM ({PREFER_GIM.upper()}) delta for {epoch} UT reference date {REFERENCE_DATE.date()}"
    plot_map(delta_maps, epoch, cbar_lims=CBAR_LIMITS_DELTA, savefig_to=fname, title=title)
    filenames.append(fname)
    sys.stdout.write(f"\rSave {fname}")    
    sys.stdout.flush()
    

2024-06-13 05:14:35,172 - SWReport - INFO - Start plotting and saving delta maps


Save gim_plot/delta_2024-05-21 00:00:00+00:00_refdate_2024-05-15.png

In [90]:
logger.info(f"Make animation out of {filenames[0]} {filenames[-1]}")
with imageio.get_writer(GIM_PLOTS_PATH / f"maps_{PREFER_GIM}.gif", mode="I", loop=5) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        sys.stdout.write(f"\rAnimate {filename}")    
        sys.stdout.flush()

2024-06-13 05:16:53,437 - SWReport - INFO - Make animation out of gim_plot/delta_2024-05-15 00:00:00+00:00_refdate_2024-05-15.png gim_plot/delta_2024-05-21 00:00:00+00:00_refdate_2024-05-15.png


Animate gim_plot/delta_2024-05-21 00:00:00+00:00_refdate_2024-05-15.png

In [91]:
from IPython.display import display, HTML
gif_path = GIM_PLOTS_PATH / f"maps_{PREFER_GIM}.gif"
html_code = f'<img src="{gif_path}" autoplay loop>'
display(HTML(html_code))